In [8]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import tensorflow as tf

from torch.utils.data import TensorDataset, DataLoader

In [9]:
# load the data
_dir = os.path.abspath('')
data_path = os.path.join(_dir, "../data/daily_cleaned.csv")
df = pd.read_csv(data_path)
df = df.drop(df.columns[0], axis=1)
new_columns = df.columns.values
new_columns[-1] = 'label'
df.columns = new_columns
print(df)

      precip  solar   air  vapor     label
0        0.0  359.6   6.2    5.0  0.000000
1        0.0  334.0   6.0    4.8  0.000000
2        0.0  347.7   6.2    5.0  0.000000
3        0.0  335.8  14.0    7.7  0.000000
4        0.4  319.7  15.6   10.0  0.000000
...      ...    ...   ...    ...       ...
2766     0.1   51.4 -16.2    1.4  0.142646
2767     0.0   59.1 -17.2    1.3  0.140948
2768     0.2   24.4 -13.7    1.9  0.139318
2769     0.1   33.7  -3.1    3.8  0.138547
2770     3.1    4.1   1.3    6.2  0.138313

[2771 rows x 5 columns]


In [10]:
# create series_to_supervised() function
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence: (t-n, ..., t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ..., t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # concatenate together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [11]:
# testing series to supervised
values = df.values
data = series_to_supervised(values)
# drop the columns we don't want to predict (predicting for current time step), so all vars at time t except var5(t)
data.drop(data.columns[[5, 6, 7, 8]], axis=1, inplace=True)
print(data)

      var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)   var5(t)
1           0.0      359.6        6.2        5.0   0.000000  0.000000
2           0.0      334.0        6.0        4.8   0.000000  0.000000
3           0.0      347.7        6.2        5.0   0.000000  0.000000
4           0.0      335.8       14.0        7.7   0.000000  0.000000
5           0.4      319.7       15.6       10.0   0.000000  0.000000
...         ...        ...        ...        ...        ...       ...
2766        0.2       49.5      -17.6        1.3   0.144958  0.142646
2767        0.1       51.4      -16.2        1.4   0.142646  0.140948
2768        0.0       59.1      -17.2        1.3   0.140948  0.139318
2769        0.2       24.4      -13.7        1.9   0.139318  0.138547
2770        0.1       33.7       -3.1        3.8   0.138547  0.138313

[2770 rows x 6 columns]


In [12]:
# split dataset into train, validation, test sets
values = data.values
train_df = values[:1386, :]
valid_df = values[1386:2079, :]
test_df = values[2079:, :]

In [13]:
# setup train data
train_x, train_y = train_df[:, :-1], train_df[:, -1] # raw numpy

# setup validation data
valid_x, valid_y = valid_df[:, :-1], valid_df[:, -1]

# setup test data
test_x, test_y = test_df[:, :-1], test_df[:, -1]

batch_size = 30

# reshape inputs (x's) to be 3D [seq_len, batch, input_size]
# using batches of 30, sequence length should always be 1
# should be (30, 1, m)
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
valid_x = valid_x.reshape((valid_x.shape[0], 1, valid_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape, test_x.shape, test_y.shape)

(1386, 1, 5) (1386,) (693, 1, 5) (693,) (691, 1, 5) (691,)


In [15]:
# begin keras training...
# let memory grow...
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# create keras model
model = Sequential()
model.add(LSTM(50, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit the model
history = model.fit(train_x, train_y, epochs=50, batch_size=30, validation_data=(valid_x, valid_y), verbose=2, shuffle=False)
# plot the history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

RuntimeError: Physical devices cannot be modified after being initialized

In [ ]:
# setup pytorch to use cuda (gpu training) if possible
# pytorch stuff
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# convert to torch tensors
train_x = torch.from_numpy(train_x).float().to(device)
train_y = torch.from_numpy(train_y).float().to(device)

valid_x = torch.from_numpy(valid_x).float().to(device)
valid_y = torch.from_numpy(valid_y).float().to(device)

test_x = torch.from_numpy(test_x).float().to(device)
test_y = torch.from_numpy(test_y).float().to(device)

print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape, test_x.shape, test_y.shape)
"""
# convert to torch datasets
# train data
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
# validation data
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
# test data
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# setup DataLoaders
train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
"""

In [7]:
# setup architecture of LSTM model
class SoilNet(nn.Module):
    def __init__(self, feature_size, output_size, hidden_size, seq_len, n_layers=2):
        super(SoilNet, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.seq_len = seq_len
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(feature_size, hidden_size, n_layers) # LSTM layer
        self.predict = nn.Linear(hidden_size, output_size) # output layer
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x.view(len(x), self.seq_len, -1), self.hidden)
        last_time_step = lstm_out.view(self.seq_len, len(x), self.hidden_size)[-1]
        y_pred = self.predict(last_time_step)
        return y_pred
    
    def init_hidden(self):
        self.hidden = (torch.zeros(self.n_layers, self.seq_len, self.hidden_size).float().to(device), torch.zeros(self.n_layers, self.seq_len, self.hidden_size).float().to(device))

In [8]:
# define arguments and instantiate model
feature_size = 5 # 5 features
output_size = 1 # just output a number
hidden_dim = 50 # size of hidden state and cell state at each time step
seq_len = 1
n_layers = 2

print(device)

model = SoilNet(feature_size, output_size, hidden_dim, seq_len, n_layers)
model = model.float()
model.to(device)

# hyperparams
lr=0.005
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

print(model)

cuda
SoilNet(
  (lstm): LSTM(5, 50, num_layers=2)
  (predict): Linear(in_features=50, out_features=1, bias=True)
)


In [ ]:
# now start the training
epochs = 10
train_hist = np.zeros(epochs)

model.train() # set to training mode
for i in range(epochs):
    model.init_hidden()
    optimizer.zero_grad()
    y_pred = model(train_x)
    loss = criterion(y_pred.float(), train_y)
    
    train_hist[i] = loss.item()
    loss.backward()
    optimizer.step()
    
    val_losses = []
    model.eval()
    
    model.train()

C:\Users\Leo\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([1386])) that is different to the input size (torch.Size([1386, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [8]:
# now check accuracy for test set...
test_losses = []
num_correct = 0

predictions = []
actuals = []

model.eval()
for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    y_pred = model(inputs.cuda())
    
    test_loss = criterion(y_pred, labels.cuda())
    test_losses.append(test_loss.item())
    pred_cpu = y_pred.cpu()
    pred_cpu = pred_cpu.detach().numpy()[:, 0]
    pred_cpu = pred_cpu.tolist()
    
    lab_cpu = labels.cpu()
    lab_cpu = lab_cpu.numpy()[:, 0]
    lab_cpu = lab_cpu.tolist()
    
    predictions.extend(pred_cpu)
    actuals.extend(lab_cpu)
    

print("Test loss: ", test_losses)
print("Mean test loss: ", np.mean(test_losses))
print("Predictions:")
print(predictions)
print()
print("Actual values:")
print(actuals)

Test loss:  [0.005716411584264437, 0.004094231013504453, 0.004811074350466154, 0.002719409148891785]
Mean test loss:  0.004335281524281708
Predictions:
[0.12422963102828122, 0.07597662783819323, 0.06288123502311305, 0.09041999476568373, 0.15030129989379576, 0.05322136544208819, 0.08266845679021903, 0.01890145984519781, 0.03230656745696325, 0.06316190664256519, 0.16552776916354603, 0.13374768384265248, 0.1483001103496152, 0.06649637069491482, 0.10380046530567676, 0.08897690126704091, 0.10861928266783023, 0.02876654099225038, 0.061777439367065964, 0.17629317428925878, 0.030901829904294804, 0.13954610532173262, 0.0732079012001161, 0.058419058777066595]

Actual values:
[0.130342251563586, 0.175548141086749, 0.0985124327956989, 0.179852501737318, 0.190332661290323, 0.169309059233449, 0.16356132392473097, 0.0461615750169722, 0.15219724462365603, 0.103205309139785, 0.132499652294854, 0.11923870532703999, 0.081446590118302, 0.0993261088709677, 0.148817126269956, 0.130387936827957, 0.1728141203